In [2]:
from platform_connector.platform_connector import platform_connector
from data_provider.data_provider import data_provider
from queue import Queue
from trading_director.trading_director import trading_director
import pandas as pd
import MetaTrader5 as mt5

In [3]:
symbols = ["EURUSD"]#,"USDJPY"]
timeframe = '1min'
#Creacion de la cola de evenetos principal
events_queue = Queue()

CONNECT = platform_connector(symbol_list=symbols)

Conexion establecida con éxito!!!
cuenta de tipo demo
Simbolo EURUSD ya estaba visible en mw
Informacion de la cuenta: 
Account ID: 10008956773
Nombre trader: Yago Garcia MARQUÉS
Broker: MetaQuotes Ltd.
Servidor: MetaQuotes-Demo
Apalancamiento: 100
Divisa: EUR
Balance: 100000.0


In [6]:

def _map_tf(timeframe: str):
        timeframe_mapping = {
            '1min': mt5.TIMEFRAME_M1,
            '2min': mt5.TIMEFRAME_M2,                        
            '3min': mt5.TIMEFRAME_M3,                        
            '4min': mt5.TIMEFRAME_M4,                        
            '5min': mt5.TIMEFRAME_M5,                        
            '6min': mt5.TIMEFRAME_M6,                        
            '10min': mt5.TIMEFRAME_M10,                       
            '12min': mt5.TIMEFRAME_M12,
            '15min': mt5.TIMEFRAME_M15,
            '20min': mt5.TIMEFRAME_M20,                       
            '30min': mt5.TIMEFRAME_M30,                       
            '1h': mt5.TIMEFRAME_H1,                          
            '2h': mt5.TIMEFRAME_H2,                          
            '3h': mt5.TIMEFRAME_H3,                          
            '4h': mt5.TIMEFRAME_H4,                          
            '6h': mt5.TIMEFRAME_H6,                          
            '8h': mt5.TIMEFRAME_H8,                          
            '12h': mt5.TIMEFRAME_H12,
            '1d': mt5.TIMEFRAME_D1,                       
            '1w': mt5.TIMEFRAME_W1,                       
            '1M': mt5.TIMEFRAME_MN1}
        try:
            return timeframe_mapping[timeframe]
        except Exception as e:
            print(f"{e}")

def get_lastest_closed_bars(symbol: str, timeframe: str,num_bars=1) -> pd.DataFrame:
        
        tf = _map_tf(timeframe)
        from_position = 1
        bars_count = num_bars if num_bars >1 else 1

        bars = mt5.copy_rates_from_pos(symbol,tf,from_position,bars_count)
        if bars is None:
            print("No se han podido cargar los datos")
            bars = pd.DataFrame()
        else:
            bars=pd.DataFrame(bars)
            bars['time'] = pd.to_datetime(bars['time'], unit='s')
            bars.set_index('time',inplace=True)
            bars.rename(columns={'tick_volume': 'tickvol', 'real_volume': 'vol'}, inplace=True)
            bars=bars[['open','high','low','close','tickvol','vol','spread']]

        if bars.empty:
            return pd.DataFrame()
        else:
            return bars

In [7]:
bars=get_lastest_closed_bars(symbol="EURUSD", timeframe = "1min" ,num_bars=1000)

In [ ]:
%%timeit
fast_avg = bars['close'].rolling(100).mean().iloc[-1]
fast_avg = bars['close'].iloc[-100:].mean()

91.6 μs ± 899 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [31]:
%%timeit
fast_avg = bars['close'].iloc[-100:].mean()

48.9 μs ± 184 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [25]:
bars

,open,high,low,close,tickvol,vol,spread
time,,,,,,,
2026-01-05 00:32:00,1.17206,1.17206,1.17206,1.17206,1,0,22
2026-01-05 00:33:00,1.17206,1.17206,1.17200,1.17201,5,0,22
2026-01-05 00:34:00,1.17201,1.17202,1.17198,1.17200,10,0,23
2026-01-05 00:35:00,1.17200,1.17201,1.17198,1.17200,16,0,21
2026-01-05 00:36:00,1.17200,1.17201,1.17199,1.17200,13,0,22
...,...,...,...,...,...,...,...
2026-01-05 17:07:00,1.16917,1.16940,1.16912,1.16931,123,0,0
2026-01-05 17:08:00,1.16931,1.16941,1.16906,1.16906,127,0,0
2026-01-05 17:09:00,1.16906,1.16909,1.16897,1.16897,156,0,0
